<a href="https://colab.research.google.com/github/hksantosh/tsai/blob/master/eva5/S4/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
  def __init__(self):
    dropout = 0.10
    super(Net, self).__init__()

    # self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF 3
    # self.pool1 = nn.MaxPool2d(2, 2)  # RF 6
    self.conv1 = nn.Sequential(
      nn.Conv2d(1, 16, 3, padding=1),  # Input: 1x28x28 Output: 16x28x28 RF: 3
      nn.ReLU(),
      nn.BatchNorm2d(16),
      nn.Dropout(dropout),
      nn.MaxPool2d(2, 2)  # Input: 16x28x28 Output: 16x14x14 RF: 6
    )

    # self.conv2 = nn.Conv2d(16, 16, 3, padding=1) # RF 8
    # self.conv3 = nn.Conv2d(16, 16, 3, padding=1)  # RF 10
    # self.pool2 = nn.MaxPool2d(2, 2)  # RF 20
    self.conv2 = nn.Sequential(
      nn.Conv2d(16, 16, 3, padding=1),  # Input: 16x14x14 Output: 16x14x14 RF: 8
      nn.ReLU(),
      nn.BatchNorm2d(16),
      nn.Dropout(dropout),
      nn.Conv2d(16, 16, 3, padding=1),  # Input: 16x14x14 Output: 16x14x14 RF: 10
      nn.ReLU(),
      nn.BatchNorm2d(16),
      nn.Dropout(dropout),
      nn.MaxPool2d(2, 2) # Input: 16x14x14 Output: 16x14x14 RF: 20
    )

    # self.conv4 = nn.Conv2d(16, 16, 3, padding=1)  # RF 22
    # self.conv5 = nn.Conv2d(16, 16, 3)   # RF 24
    # self.conv6 = nn.Conv2d(16, 16, 3)   # RF 26
    self.conv3 = nn.Sequential(
      nn.Conv2d(16, 16, 3, padding=1),  # Input: 16x7x7 Output: 16x7x7 RF: 22
      nn.ReLU(),
      nn.BatchNorm2d(16),
      nn.Dropout(dropout),
      nn.Conv2d(16, 16, 3),  # Input: 16x7x7 Output: 16x5x5 RF: 24
      nn.ReLU(),
      nn.BatchNorm2d(16),
      nn.Dropout(dropout),
      nn.Conv2d(16, 32, 3),  # Input: 16x5x5 Output: 16x3x3 RF: 26
      nn.ReLU(),
      nn.BatchNorm2d(32),
      nn.Dropout(dropout)
    )

    # self.conv7 = nn.Conv2d(16, 10, 3)
    self.conv4 = nn.Sequential(
      nn.Conv2d(32, 10, 3),  # Input: 16x3x3 Output: 10x1x1 RF: 28
      nn.ReLU()
    )


  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           2,320
              ReLU-7           [-1, 16, 14, 14]               0
       BatchNorm2d-8           [-1, 16, 14, 14]              32
           Dropout-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]           2,320
             ReLU-11           [-1, 16, 14, 14]               0
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    print(f'Epoch #{epoch}')
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch #1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0401, Accuracy: 9876/10000 (98.76%)

Epoch #2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0214, Accuracy: 9938/10000 (99.38%)

Epoch #19


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0231, Accuracy: 9929/10000 (99.29%)

Epoch #20


100%|██████████| 938/938 [00:18<00:00, 50.85it/s]



Test set: Average loss: 0.0189, Accuracy: 9944/10000 (99.44%)

